### Importamos librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Importamos nuestros datos a un df

In [ ]:
df = pd.read_csv(r'AccidentesAviones.csv')

### Exploración Inicial

In [ ]:
df.head()

In [ ]:
#Vemos la dimensión de nuestro dataset
df.shape

In [ ]:
#Corroboramos existencia de valores nulos
df.isnull().sum()

A simple vista parece que no existen valores nulos, pero estos estan expresados con el valor '?' así que luego veremos en concreto esos datos.

In [ ]:
#Vemos los tipos de datos correspondientes a nuestros campos
df.info()

### Renombramos columnas

In [ ]:
df = df.rename(columns={
    'fecha': 'Fecha',
    'HORA declarada': 'Hora_accidente', 
    'flight_no': 'Nro_vuelo',
    'route': 'Route',
    'registration': 'Registro',
    'all_aboard': 'Total_a_bordo',
    'OperadOR': 'Operador', 
    'ac_type': 'Tipo_aeronave',
    'PASAJEROS A BORDO': 'Pasajeros_a_bordo', 
    'cantidad de fallecidos': 'Total_fallecidos',
    'passenger_fatalities': 'Pasajeros_fallecidos',
    'crew_fatalities': 'Tripulacion_fallecida',
    'crew_aboard': 'Tripulacion_a_bordo',
    'ground': 'Muertos_tierra',
    'summary': 'Resumen_accidente',
    })

### Dataset consiste en 5008 observaciones (crashes) y 18 atributos, estos son:

*Fecha*: fecha en que se produjo el accidente

*Hora_accidente*: hora que se produjo el accidente

*Ruta*: lugar donde se produjo el accidente

*Operador*: aerolínea de la aeronave

*Nro_vuelo*: número de vuelo

*Route*: ruta tomada en avión

*Tipo_aeronave*: tipo de aeronave

*Registro* -

*cn_ln* - 

*Total_a_bordo*: total personas a bordo

*Pasajeros_a_bordo*: número de pasajeros a bordo

*Triplulacion_a_bordo*: número de tripulación a bordo

*Total_fallecidos*: número total de personas fallecidas

*Pasajeros_fallecidos*: número de pasajeros fallecidos

*Tripulacion_fallecida*: número de tripulación fallecida

*Ground*: número de personas muertas en tierra

*Resumen_accidente*: breve resumen del caso



### Tratamiento de datos

In [ ]:
# Eliminar la primer columna ya que es irrelevante para nuestro análisis (enumera filas)
df = df.drop('Unnamed', axis=1)

In [ ]:
df.head(1)

In [ ]:
#Cambiamos los datos '?' por valores acorde a cada campo, ya que deduje que son nulos
df['Fecha'] = df['Fecha'].replace('?', np.NaN)
df['Hora_accidente'] = df['Hora_accidente'].replace('?', np.NaN)
df['Ruta'] = df['Ruta'].replace('?', 'Sin dato')
df['Operador'] = df['Operador'].replace('?', 'Sin dato')
df['Nro_vuelo'] = df['Nro_vuelo'].replace('?', 'Sin dato')
df['Route'] = df['Route'].replace('?', 'Sin dato')
df['Tipo_aeronave'] = df['Tipo_aeronave'].replace('?', 'Sin dato')
df['Registro'] = df['Registro'].replace('?', 'Sin dato')
df['cn_ln'] = df['cn_ln'].replace('?', 'Sin dato')
df['Total_a_bordo'] = df['Total_a_bordo'].replace('?', 0)
df['Pasajeros_a_bordo'] = df['Pasajeros_a_bordo'].replace('?', 0)
df['Tripulacion_a_bordo'] = df['Tripulacion_a_bordo'].replace('?', 0)
df['Total_fallecidos'] = df['Total_fallecidos'].replace('?', 0)
df['Pasajeros_fallecidos'] = df['Pasajeros_fallecidos'].replace('?', 0)
df['Tripulacion_fallecida'] = df['Tripulacion_fallecida'].replace('?', 0)
df['Muertos_tierra'] = df['Muertos_tierra'].replace('?', 0)
df['Resumen_accidente'] = df['Resumen_accidente'].replace('?', 'Sin dato')

In [ ]:
df.isnull().sum()

### Cambios en columnas

In [ ]:
#Cambiamos el tipo de dato de 'fecha'
df['Fecha'] = pd.to_datetime(df['Fecha'])

In [ ]:
#Convertimos las columnas a tipo numérico
a_entero = ['Total_a_bordo', 'Pasajeros_a_bordo', 'Tripulacion_a_bordo', 'Total_fallecidos', 'Pasajeros_fallecidos', 'Tripulacion_fallecida', 'Muertos_tierra']
df[a_entero] = df[a_entero].astype(int)

In [ ]:
#Cambiamos el formato de la columna 'hora declarada'
df['Hora_accidente'] = df['Hora_accidente'].str.slice(0, 2) + ':' + df['Hora_accidente'].str.slice(2)
df['Hora_accidente'] = df['Hora_accidente'].str.replace("c ", "")

Corroboramos los cambios de formato, tipo y si quedan valores nulos aún

In [ ]:
df.info()

Creación de nuevas columnas para su posterior visualización

In [ ]:
# Creamos nuevas columna con la tasa de mortalidad, tasa de supervivencia y número de sobrevivientes
df['Tasa_mortalidad'] = (df['Total_fallecidos'] / df['Total_a_bordo']) * 100
df['Tasa_supervivencia'] = ((df['Total_a_bordo'] - df['Total_fallecidos']) / df['Total_a_bordo']) * 100
df['Sobrevivientes'] = df['Total_a_bordo'] - df['Total_fallecidos']
#df['Anio'] = pd.DatetimeIndex(df['Fecha']).year



In [ ]:
df.head(1)

Cantidad de accidentes por año

In [ ]:
# Calcular la cantidad de accidentes por año
accidentes_por_anio = df.groupby(df['Fecha'].dt.year).size()

# Seleccionar solo algunos años para mostrar en el gráfico
seleccion_anios = accidentes_por_anio.index[::5]  # Cambiar 5 por el intervalo deseado

# Crear el gráfico de líneas
plt.figure(figsize=(10, 6))
plt.plot(accidentes_por_anio.index, accidentes_por_anio.values, marker='o')
plt.xlabel('Año')
plt.ylabel('Cantidad de accidentes')
plt.title('Cantidad de accidentes por año')

# Mostrar solo los años seleccionados en el eje x
plt.xticks(seleccion_anios)

plt.tight_layout()
plt.show()


¿Hubo más de un accidente en el mismo día?

In [ ]:
df['Fecha'].value_counts()[:6]

In [ ]:
#20 fechas con mas accidentes aereos
accidentes_20 = df['Fecha'].value_counts().head(30)

# Crear el gráfico de barras
plt.figure(figsize=(10, 6)) 
accidentes_20.plot(kind='bar')

# Personalizar el gráfico
plt.title('20 días con mayor número de accidentes')
plt.xlabel('Fechas')
plt.ylabel('Frecuencia')

# Mostrar el gráfico
plt.show()


Las dos fechas con más accidentes en la historia son: 31-08-1988 y 11-09-2001
El 11 de septiembre de 2001 se puede relacionar con los ataques terroristas en Estados Unidos, donde varios aviones comerciales fueron secuestrados y utilizados como armas.

No es raro tener varios (hasta 4) accidentes por día. A partir del 11 de septiembre, hubo 5 fechas más en las que hubo 4 accidentes por día. Parecen una coincidencia aparte del 28-02-1973, cuando en Polonia hubo 2 accidentes por mal tiempo.

**Visializaremos cómo ha cambiado la tasa de supervivencia media a lo largo de los años.**

In [ ]:
tasa_supervivencia_media_por_anio = df.groupby(df['Fecha'].dt.year)['Tasa_supervivencia'].mean()

plt.figure(figsize=(10, 6))
plt.plot(tasa_supervivencia_media_por_anio.index, tasa_supervivencia_media_por_anio.values, marker='o')
plt.xlabel('Año')
plt.ylabel('Tasa de supervivencia media')
plt.title('Tasa de supervivencia media por año')
plt.xticks([x for x in range(1908,2009,10)], rotation='vertical')
plt.axhline(y=data_mean, color='r', linestyle='-')

plt.tight_layout()
plt.show()

 **tasa de mortalidad**

In [ ]:
# Calcular la tasa de mortalidad promedio por año
tasa_mortalidad_promedio_por_anio = df.groupby(df['Fecha'].dt.year)['Tasa_mortalidad'].mean()

# Calcular la media de la tasa de mortalidad
media_tasa_mortalidad = df['Tasa_mortalidad'].mean()

# Crear el gráfico de línea
plt.figure(figsize=(10, 6))
plt.plot(tasa_mortalidad_promedio_por_anio.index, tasa_mortalidad_promedio_por_anio.values, marker='o')
plt.axhline(media_tasa_mortalidad, color='r', linestyle='-', label='Media')
plt.xlabel('Año')
plt.ylabel('Tasa de mortalidad promedio')
plt.title('Tasa de mortalidad promedio por año')
plt.legend()

plt.tight_layout()
plt.show()


### Explorando las operadoras


¿Qué operador tiene más aviones estrellados?

In [ ]:
df['Operador'].value_counts()

¿Qué operador tiene más fatalidades?

In [ ]:
df.groupby(['Operador'])['Total_fallecidos'].sum().sort_values( ascending=False).head(20).plot.bar()

*La aerolinea con el mayor número de accidentes es Aeroflot, seguido de Military U.S Air force*

Analizando en particular a la aerolinea *Aeroflot*

In [ ]:
# Filtrar los accidentes de la aerolínea "Aeroflot"
filtro_aeroflot = df['Operador'] == "Aeroflot"

# Agrupar los accidentes de "Aeroflot" por año y contar la cantidad de accidentes por año
accidentes_aeroflot_por_anio = df.loc[filtro_aeroflot].groupby(df['Fecha'].dt.year).size()

# Crear el gráfico de barras
plt.figure(figsize=[12, 6])
accidentes_aeroflot_por_anio.plot(kind='bar')
plt.xlabel('Año')
plt.ylabel('Cantidad de accidentes')
plt.title('Accidentes de Aeroflot por año')

plt.tight_layout()
plt.show()


¿Cuál es el peor accidente aéreo (mayor fatalidad) para cada operador?

In [ ]:
df.groupby(['Operador'])['Total_fallecidos'].max().sort_values( ascending=False)

Los dos accidentes más grandes por número de muertes incluyen factores humanos. Puede ver los detalles a continuación.

In [ ]:
id_of_max_Fatalities = df['Total_fallecidos'].sort_values().dropna().index

In [ ]:
df.loc[id_of_max_Fatalities[-1]].Resumen_accidente

In [ ]:
df.loc[id_of_max_Fatalities[-2]].Resumen_accidente

¿En qué accidente sobrevivió la mayoría de las personas?

In [ ]:
max_survive_index = df['Sobrevivientes'].idxmax()
df.loc[max_survive_index]


¿Cuál fue la causa de este accidente?

In [ ]:
df['Resumen_accidente'].loc[max_survive_index]

In [ ]:
df.to_csv('df_final.csv', index=False)